In [ ]:
import torch 
####
#TODO

# aussi il y a un biais au debut quil faudrait corriger en fonction de la loneur du path
# peut etre qu il faudrait calculer le nb de possibile paths
# reduire le sampling 

# faire symbolic
# warnings
# initialiser h0 pour rontrainer
# check if sub and divide takes x0 and x1 in the good order
# remove is_none I think it's useless 

import numpy as np
from Loss import Loss
from RON import RecurrentOccamNet
from RONTrainer import RONTrainer
from cocob import COCOBBackprop 
from bases import *

X = [[np.array([np.cos(i), 1/100], dtype=np.float64)] for i in range(5)]

Y = []
W = []
w = np.array([1., 1.])
for x in X:
    y = np.sin(np.dot(x[0],w))
    w += x[0]*y
    Y.append([y])
    W.append(np.array(w.copy()))


In [ ]:
r_model = RecurrentOccamNet([2], 2, [1], 
                            [[on_sin(), on_cos(), on_sum(), on_dot(), on_mul()],
                             [on_sin(), on_cos(), on_sum(), on_dot(), on_mul()],
                             [on_sin(), on_cos(), on_sum(), on_dot(), on_mul()], 
                            [on_identity(), on_identity()]], 
                            [0.01, 0.01, 0.01, 0.01], 
                            skip_connections=True, is_none=False)
criterion = Loss(0.01)
optimizer = COCOBBackprop(r_model.parameters())
trainer = RONTrainer(r_model, criterion, optimizer)

trained_model = trainer.fit(X, Y, 4, 3)

In [ ]:
list(trained_model.parameters())

In [ ]:
trained_model.samples_paths_and_probas(3)


In [ ]:
import copy
nb_sampled_paths = 1

#X = [x.append(np.array([None])) for x in X]
X = [[copy.deepcopy(x) for _ in range(nb_sampled_paths)] for x in X]
# Reshape Y so that shape == (nb elements, output dim).
Y = torch.tensor(Y, requires_grad = False)

the_paths, the_keys_probas, the_keys_path2 = trained_model.samples_paths_and_probas(nb_sampled_paths)

for x, y in zip(X, Y):
    # This imply to compute the pred of the model for each sampled path
    pred = trained_model.forward(x, the_paths) # shape == (nb sampled paths, output dim)
    print(pred, y)

In [ ]:
print(trained_model.forward(X, the_path)[1])
print(W[:n])